In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [2]:
word_index = imdb.get_word_index()
reverse_word_index = {value:key for key,value in word_index.items()}


In [3]:
model = load_model('simple_rnn_imdb.h5')

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [5]:
def decode(encoded_review):
    return ' '.join([reverse_word_index.get(i-3,'?')for i in encoded_review])

In [8]:
import re

In [15]:
#function to pre-process user input
def preprocess_text(text):
    words = text.lower()
    words = re.sub(r'[^\w\s]','',words) #removes punctuation
    words = words.split()
    encoded_review = [word_index.get(word,2)+3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review],maxlen=500) #encoded review should be wrapped in a list [] as parameter, else it will give error
    return padded_review
# padding is done coz deep learning models like rnn, lstm require fixed length input

In [16]:
#prediction function
def predict_sentiment(review):
    preprocessed_review = preprocess_text(review)
    prediction = model.predict(preprocessed_review)
    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    return sentiment,prediction[0][0]
#prediction return 2d array, pred[0][0] extracts actual float value from 2d array






In [17]:
#example review for prediction 
example_review = 'The movie was fantastic, the cast was great and the story was thrilling.'

In [18]:
sentiment,score = predict_sentiment(example_review)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step


In [22]:
print(f"REVIEW: {example_review}")
print(f"\nsentiment: {sentiment}")
print(f"\nscore: {round(float(score),2)}") #score was np.float so converted into normal float first then applied round function

REVIEW: The movie was fantastic, the cast was great and the story was thrilling.

sentiment: Positive

score: 0.71
